In [8]:
import pandas as pd
import pandas_ta
import ta

In [9]:
df = pd.read_csv('../BTC_ETH_1week_1m.csv')

In [10]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std*nstd
    lower_band = sma - std*nstd
    return upper_band, lower_band

ta.volatility.AverageTrueRange
symbols = ['BTC']


nstd = 3

for symbol in symbols:
    df[f'{symbol}_SMA'] = sma(df[f'{symbol}-USD_Open'], 20)

    df[f'{symbol}_SMA_200'] = sma(df[f'{symbol}-USD_Open'], 200)

    df[f'{symbol}_Upper_Band'], df[f'{symbol}_Lower_Band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_SMA'], 20, nstd)
    
    df[f'{symbol}_ATR'] = ta.volatility.average_true_range(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], window=14)
    
    dfs = pandas_ta.supertrend(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], period=10, length=None, multiplier=3)
    df[f'{symbol}_SuperDir'] = dfs['SUPERTd_7_3.0']

    df[f'{symbol}_RSI'] = ta.momentum.rsi(close=df[f'{symbol}-USD_Close'], window=20)

df.dropna(inplace=True)

In [11]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.short_unit = None
        self.short_amount = 0
        self.short_price = 0
        self.long_price = 0

        self.sl = 0
        self.max_inner_buy = .35
        self.band_stop_run = 0.05    # <= 1
        self.slm = 3



        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.symbols = symbols
        
        self.buys = []
        self.sells = []
        
        self.wins = 0
        self.losses = 0
        self.ties = 0
        
        self.trading_fee_multiplier = 0.9996      # VIP level 0, paying fees with BNB 0.99975
        self.number_of_buys = len(self.buys)

        self.cross_up = {}
        self.cross_down = {}
        self.sma_cross_up = {}
        self.sma_cross_down = {}
        self.reset_crosses([symbols])

    def buy_long(self, symbol, long_price, atr, time):
        self.balance_amount = (self.balance_amount / long_price) * self.trading_fee_multiplier
        self.long_price = long_price
        self.balance_unit = symbol
        self.buys.append([symbol, time, long_price, 'long'])
        self.number_of_buys = len(self.buys)
        self.sl = long_price - (env.slm*atr)
        print('\n\n', 'filled long', long_price, time)

    def buy_short(self, symbol, short_price, atr, time):
        self.balance_amount = self.balance_amount * self.trading_fee_multiplier
        self.short_amount = (self.balance_amount / short_price) 
        self.short_price = short_price
        self.short_unit = symbol
        self.buys.append([symbol, time, short_price, 'short'])
        self.number_of_buys = len(self.buys)
        self.sl = short_price + (self.slm*atr)
        print('\n\n', 'filled short', short_price, time)

    def sell_long(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'
    
    def sell_short(self, sell_price, time):
        self.balance_amount = (self.balance_amount + (self.short_amount * ((self.short_price) - sell_price*self.trading_fee_multiplier)))
        self.sells.append([symbol, time, sell_price])
        self.short_unit = None
        self.short_amount = 0
    

    def reset_crosses(self, symbols):
        for symbol in self.symbols:
            self.cross_up[symbol] = False
            self.cross_down[symbol] = False
            self.sma_cross_up[symbol] = False
            self.sma_cross_down[symbol] = False
    
    def good_buy(self, type, bbhigh, bblow, close):
        bbrange = bbhigh - bblow
        maximum = 0
        if type == 'long':
            maximum = bblow + (bbrange * self.max_inner_buy)
        
        if type == 'short':
            maximum = bbhigh - (bbrange * self.max_inner_buy)
        
        return maximum



In [12]:
"""
BUY:
    Long: 
        1. Check if symbol crosses under lower boll. band
        2. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Buy long

    Short:
        1. Check if symbol crosses over upper boll. band
        2. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Buy short

SELL:
    Long:
        1. Check if symbol crosses over upper boll. band
        2. Check if close is under stop loss; if so, sell
        3. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Sell long
    
    Short:
        1. Check if symbol crosses under lower boll. band
        2. Check if close goes over stop loss; if so, sell
        3. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Sell short
"""

'\nBUY:\n    Long: \n        1. Check if symbol crosses under lower boll. band\n        2. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Buy long\n\n    Short:\n        1. Check if symbol crosses over upper boll. band\n        2. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Buy short\n\nSELL:\n    Long:\n        1. Check if symbol crosses over upper boll. band\n        2. Check if close is under stop loss; if so, sell\n        3. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Sell long\n    \n    Short:\n        1. Check if symbol crosses under lower boll. band\n        2. Check if close goes over stop loss; if so, sell\n        3. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Sell short\n'

In [13]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD', symbols=symbols)
for i in range(len(df)):    
    print(env.balance_amount, env.balance_unit, env.number_of_buys)
    if env.balance_unit == 'BUSD' and env.short_unit == None:  # Want to buy
        for symbol in symbols:

            ######## LONG           (lower + ((upper-lower)*multiplier))
            if env.cross_down[symbol] == True and (df[f'{symbol}-USD_Close'].iloc[i] <= env.good_buy('long', df[f'{symbol}_Upper_Band'].iloc[i], df[f'{symbol}_Lower_Band'].iloc[i], df[f'{symbol}-USD_Close'].iloc[i])) and (df[f'{symbol}-USD_Close'].iloc[i] >= df[f'{symbol}_SMA_200'].iloc[i]): #and df[f'{symbol}_SuperDir'].iloc[i] == 1 and df[f'{symbol}_RSI'].iloc[i] <= 30
                env.buy_long(symbol, long_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i],time=df['OpenTime'].iloc[i])
                env.reset_crosses(symbols)
                break
            
            if df[f'{symbol}-USD_Low'].iloc[i] <= df[f'{symbol}_Lower_Band'].iloc[i]:
                env.cross_down[symbol] = True
    

            ######## SHORT
            if env.cross_up[symbol] == True and (df[f'{symbol}-USD_Close'].iloc[i] >= env.good_buy('short', df[f'{symbol}_Upper_Band'].iloc[i], df[f'{symbol}_Lower_Band'].iloc[i], df[f'{symbol}-USD_Close'].iloc[i]) and (df[f'{symbol}-USD_Close'].iloc[i] <= df[f'{symbol}_SMA_200'].iloc[i])): #and df[f'{symbol}_SuperDir'].iloc[i] == -1 and df[f'{symbol}_RSI'].iloc[i] >= 70
                env.buy_short(symbol, short_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i], time=df['OpenTime'].iloc[i])
                env.reset_crosses(symbols)
                break
            
            if df[f'{symbol}-USD_High'].iloc[i] >= df[f'{symbol}_Upper_Band'].iloc[i]:
                env.cross_up[symbol] = True



    ####################################################################################################################

    ############## Want to sell long position
    elif env.balance_unit != 'BUSD':

        # Sell long for profit
        if env.cross_up[env.balance_unit] == True and df[f'{env.balance_unit}-USD_Close'].iloc[i] <= (df[f'{env.balance_unit}_Upper_Band'].iloc[i] - ((df[f'{env.balance_unit}_Upper_Band'].iloc[i]-df[f'{env.balance_unit}_Lower_Band'].iloc[i]) * env.band_stop_run)):
            print('sell long', df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell long at stop loss
        elif df[f'{env.balance_unit}-USD_Close'].iloc[i] <= env.sl:
            print('Stop loss hit: sell long', df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.losses += 1
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell long at break-even  
        if (env.sma_cross_up[env.balance_unit] == True) and (df[f'{env.balance_unit}-USD_Low'].iloc[i] <= env.long_price*((1/env.trading_fee_multiplier)**2)) and (df[f'{env.balance_unit}-USD_High'].iloc[i] >= env.long_price*((1/env.trading_fee_multiplier)**2)):
            print('Break even hit: sell long', env.long_price*((1/env.trading_fee_multiplier)**2), df['OpenTime'].iloc[i], '\n\n')
            env.ties += 1
            env.sell_long(sell_price=env.long_price*((1/env.trading_fee_multiplier)**2), time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue

        # Check if SMA is crossed up
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= df[f'{env.balance_unit}_SMA'].iloc[i]:
            print('SMA long crossed')
            env.sma_cross_up[env.balance_unit] = True

        # Check if upper band is crossed
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= df[f'{env.balance_unit}_Upper_Band'].iloc[i]:
            print('long crossed')
            env.cross_up[env.balance_unit] = True
    

    ############## Want to sell short position
    else:
        print(env.short_amount)
        # Sell short for profit
        if env.cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] + ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
            print('sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell short at stop loss
        elif df[f'{env.short_unit}-USD_Close'].iloc[i] >= env.sl:
            print('Stop loss hit: sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.losses += 1
            env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell short at break-even  
        if env.sma_cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_High'].iloc[i] >= env.short_price*(env.trading_fee_multiplier**2) and df[f'{env.short_unit}-USD_Low'].iloc[i] <= env.short_price*(env.trading_fee_multiplier**2):
            print('Break even hit: sell short', env.short_price*((env.trading_fee_multiplier)**2), df['OpenTime'].iloc[i], '\n\n')
            env.ties += 1
            env.sell_short(sell_price=env.short_price*((env.trading_fee_multiplier)**2), time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue

        # Check if SMA is crossed down
        if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_SMA'].iloc[i]:
            print('SMA short crossed')
            env.sma_cross_down[env.short_unit] = True

        # Check if lower band is crossed
        if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_Lower_Band'].iloc[i]:
            print('short crossed')
            env.cross_down[env.short_unit] = True





if env.balance_unit != 'BUSD':
    env.sell_long(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
if env.short_unit != None:
    env.sell_short(df[f'{env.short_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.number_of_buys, env.wins, env.ties, env.losses)

100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0


 filled long 38445.8 2021-01-14 16:15:00
0.002600023929792071 BTC 1
0.002600023929792071 BTC 1
0.002600023929792071 BTC 1
0.002600023929792071 BTC 1
0.002600023929792071 BTC 1
0.002600023929792071 BTC 1
0.002600023929792071 BTC 1
0.002600023929792071 BTC 1
SMA long crossed
0.002600023929792071 BTC 1
0.002600023929792071 BTC 1
SMA long crossed
0.002600023929792071 BTC 1
SMA long crossed
0.002600023929792071 BTC 1
SMA long crossed
0.00

In [14]:

data = pd.DataFrame([(i[1] for i in env.buys), (j[2] for j in env.buys), (a[3] for a in env.buys), (k[1] for k in env.sells), (l[2] for l in env.sells)])
data

,0,1,2,3,4,5,6,7,8,9,...,454,455,456,457,458,459,460,461,462,463
0,2021-01-14 16:15:00,2021-01-15 07:45:00,2021-01-16 03:45:00,2021-01-17 10:30:00,2021-01-18 02:00:00,2021-01-19 08:00:00,2021-01-19 14:00:00,2021-01-20 18:30:00,2021-01-21 14:30:00,2021-01-22 08:30:00,...,2021-12-23 07:45:00,2021-12-24 10:00:00,2021-12-25 03:15:00,2021-12-25 06:15:00,2021-12-26 05:15:00,2021-12-26 10:30:00,2021-12-27 03:00:00,2021-12-27 15:45:00,2021-12-30 06:45:00,2021-12-31 08:45:00
1,38445.8,37750.9,37455.6,35901.1,35899.3,36883.2,36660.7,35636.1,32633.1,32245.7,...,48619.0,50688.5,50838.0,50757.0,50036.1,50128.6,50851.9,51239.4,47505.7,47943.2
2,long,long,short,short,short,long,long,short,short,short,...,short,long,long,long,short,short,long,long,short,long
3,2021-01-14 22:45:00,2021-01-15 10:00:00,2021-01-16 07:45:00,2021-01-17 15:45:00,2021-01-18 06:15:00,2021-01-19 10:00:00,2021-01-19 17:45:00,2021-01-21 04:30:00,2021-01-21 17:30:00,2021-01-22 11:15:00,...,2021-12-23 12:00:00,2021-12-24 14:15:00,2021-12-25 04:45:00,2021-12-25 08:00:00,2021-12-26 07:00:00,2021-12-26 16:15:00,2021-12-27 05:45:00,2021-12-27 19:00:00,2021-12-30 10:15:00,2021-12-31 09:45:00
4,38476.575104,36305.5,37425.641513,35872.384864,35870.586304,36912.724273,36690.046167,32946.2,30574.8,32219.908599,...,49140.0,51499.2,50878.694815,50797.629976,49831.5,50406.0,50892.605942,50586.4,47467.703041,47981.577585
